# Setup

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from random import randint
import math
from datetime import datetime
import pickle

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
os.getcwd()

'D:\\Github\\Stomach-Status-Classification\\Experiment\\Approach\\DecisionTree'

In [3]:
for i in range(3):
    os.chdir("..")

In [4]:
os.getcwd()

'D:\\Github\\Stomach-Status-Classification'

In [5]:
main_data_dir = os.getcwd() + "\\Data set"
kmean_data_dir = main_data_dir + "\\kmean_dataset"
kmean_data_10cv_dir = kmean_data_dir + "\\10cv_64"
import glob

kmean_data_10cv_lst = [x for x in glob.glob(kmean_data_10cv_dir + '\\*') if 'orb' in x and 'index' in x]
print(kmean_data_10cv_lst)

['D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index0.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index1.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index2.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index3.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index6.csv', 'D:\\Github\\Stomach-Status-Classification\\Dat

In [6]:
kmean_train_paths = [x for x in kmean_data_10cv_lst if 'train' in x]
kmean_test_paths = [x for x in kmean_data_10cv_lst if 'test' in x]
print(kmean_train_paths)
print()
print(kmean_test_paths)
print(len(kmean_train_paths))
print(len(kmean_test_paths))

['D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index0.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index1.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index2.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index3.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index6.csv', 'D:\\Github\\Stomach-Status-Classificati

In [7]:
def to_categorical(y, num_classes=None, dtype='float32'):
    """to_categorical _summary_

    Arguments:
        y -- The label of the data set with the shape of [None, 1]

    Keyword Arguments:
        num_classes -- The num_classes in the data set (default: {None})
        dtype -- the type of each element of the label after reshape (default: {'float32'})

    Returns:
        the label of the data set with the shape of [number of samples, number of classes].
    """    
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [8]:
class roc_curve_score:
    def __init__(self, y_true, y_pred, num_class):
        self.y_test = to_categorical(y_true)
        self.y_score = to_categorical(y_pred)
        print(np.unique(self.y_test), np.unique(self.y_score))
        self.fprs = {}
        self.tprs = {}
        self.thresh_holds = {}
        for x in range(num_class):
            self.fprs[x], self.tprs[x], self.thresh_holds[x] = roc_curve(self.y_test[:, x], self.y_score[:, x], drop_intermediate=False)
            self.fprs[x] = self.fprs[x].tolist()
            self.tprs[x] = self.tprs[x].tolist()
            self.thresh_holds[x] = self.thresh_holds[x].tolist()
        
        self.fpr_micro_avg, self.tpr_micro_avg, self.threshold_micro_avg, = roc_curve(self.y_test.ravel(), self.y_score.ravel())
        
        all_fpr = np.unique(np.concatenate([self.fprs[i] for i in range(num_class)]))
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(num_class):
            mean_tpr += np.interp(all_fpr, self.fprs[i], self.tprs[i])
        mean_tpr /= num_class
        self.fpr_macro_avg = all_fpr.tolist()
        self.tpr_macro_avg = mean_tpr.tolist()
    
    def get_tpr(self, _class = None):
        if(_class):
            return self.tprs[_class]
        else:
            return self.tprs
    
    def get_fpr(self, _class = None):
        if(_class):
            return self.fprs[_class]
        else:
            return self.fprs

    def get_thresholds(self, _class = None):
        if(_class):
            return self.thresh_holds[_class]
        else:
            return self.thresh_holds

    def get_roc_dict(self):
        return {
            "tpr" : self.get_tpr(),
            "fpr" : self.get_fpr(),
            "thresholds" : self.get_thresholds(),
            "fpr_micro_avg" : self.fpr_micro_avg.tolist(),
            "tpr_micro_avg" : self.tpr_micro_avg.tolist(),
            "fpr_macro_avg" : self.fpr_macro_avg,
            "tpr_macro_avg" : self.tpr_macro_avg,
        }

In [9]:
class utils:
    def __init__(self, confusion_matrix, y_true, y_pred):
        """
            - confusion_matrix: 2x2 numpy array
            - y_true: array of label
            - y_pred: array of output value calculated by model
            - fold_count: number of folds
        """
        
        # Initilize all indicator
        self.TP = confusion_matrix[0][0] # true positive
        self.FN = confusion_matrix[1][0] # false negative
        self.FP = confusion_matrix[1][0] # false positive
        self.TN = confusion_matrix[1][1] # true negative
        self.precision = self.TP/(self.FN +  self.TP) # Precision Score - Positive Predictive Value
        self.recall = self.TP / (self.TP + self.FN)
        self.f1_score =  (2 * self.TP) / (2 * self.TP + self.FP + self.FN)
        self.sensitivity = self.TP / (self.TP + self.FN) # True Positive Rate
        self.specificity = self.TN / (self.TN + self.FP) # False Positive Rate
        self.negative_predictive_value = self.TN / (self.TN + self.FN) # Negative Predictive Value
        self.false_negative_rate = self.FN / (self.FN + self.TP) # False Negative Rate
        self.false_positive_rate = self.FP / (self.FP + self.TN) # False Positive Rate
        self.false_discovery_rate = self.FP / (self.FP + self.TP) # False Discovery Rate
        self.false_omission_rate = self.FN / (self.FN + self.TN) # False Ommision Rate
        self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
        self.negative_likelihood_ratio = self.false_negative_rate / self.specificity # Negative Likelihood Ratio
        self.prevalence_threshold = math.sqrt(self.false_positive_rate) / (math.sqrt(self.sensitivity) + 
                                                                      math.sqrt(self.false_positive_rate)) # Prevalance Threshold
        self.threat_score = self.TP / (self.TN + self.FN + self.FP) # Threat Score
        self.prevalence = (self.TP + self.FN)/(self.TP + self.FN + self.TN + self.FP) # Prevalance 
        #  Matthews correlation coefficient
        self.matthews_correlation_coefficient = (self.TP*self.TN - self.FN*self.FP) / ((self.TP + self.FP)
                                                                                       *(self.TP + self.FN)
                                                                                       *(self.TN + self.FP)
                                                                                       *(self.TN + self.FN))
        self.fowlkes_mallows_index = math.sqrt(self.sensitivity + self.precision) # Fowlkes–Mallows index
        self.informedness = self.sensitivity + self.specificity - 1 # informedness
        self.markedness = self.precision + self.negative_predictive_value - 1 # markedness
        self.diagnostic_odds_ratio = self.positive_likelihood_ratio / self.negative_likelihood_ratio # Diagnostic odds ratio
        self.accuracy = (self.TP + self.TN) / (self.TP + self.TN + self.FP + self.FN)
        self.balanced_accuracy = (self.sensitivity + self.specificity) / 2
        self.roc_auc_macro = roc_auc_score(y_true, y_pred)
        self.roc_auc_micro = roc_auc_score(y_true, y_pred, average = 'micro')
        self.roc_auc_weighted = roc_auc_score(y_true, y_pred, average = 'weighted')
        self.cls_report = classification_report(y_true, y_pred)
        
        # Initilize the structure of output_dicts
        self.confusion_matrix = {
            "TP" : self.TP,
            "TN" : self.TN,
            "FN" : self.FN,
            "FP" : self.FP,
            "precision" : self.precision,
            "recall" : self.recall,
            "f1_score" : self.f1_score,
            "sensitivity" : self.sensitivity,
            "specificity" : self.specificity,
            "negative_predictive_value" : self.negative_predictive_value,
            "false_negative_rate" : self.false_negative_rate,
            "false_positive_rate" : self.false_positive_rate,
            "false_discovery_rate" : self.false_discovery_rate,
            "false_omission_rate" : self.false_omission_rate,
            "Positive_likelihood_ratio" : self.positive_likelihood_ratio,
            "Negative_likelihood_ratio" : self.negative_likelihood_ratio,
            "prevalence_threshold" : self.prevalence_threshold,
            "threat_score" : self.threat_score,
            "Prevalence" : self.prevalence,
            "Matthews_correlation_coefficient" : self.matthews_correlation_coefficient,
            "Fowlkes_Mallows_index" : self.fowlkes_mallows_index,
            "informedness" : self.informedness,
            "markedness" : self.markedness,
            "Diagnostic_odds_ratio" : self.diagnostic_odds_ratio,
            "accuracy" : self.accuracy,
            "balanced_accuracy" : self.balanced_accuracy
        }
        
        self.roc_auc_score = {
            "Macro": self.roc_auc_macro, 
            "Micro": self.roc_auc_micro,
            "Weight": self.roc_auc_weighted
        }
        
#         self.roc_curve = roc_curve_score(y_true=y_true, y_pred=y_pred, num_class=2).get_roc_dict()
        
        self.sub_dict = {
            "Confusion Matrix" : self.confusion_matrix,
            "ROC_AUC_SCORE" : self.roc_auc_score,
            "Classification Report" : classification_report(y_true.tolist(), y_pred.tolist(), 
                                                            labels = [0, 1], # 0 : Licit, 1 : Illicit
                                                            output_dict = True),
#             "ROC_DRAW" : self.roc_curve
        }
    def get_value(self):
        return self.sub_dict
        
# Test 
y_true = np.array([randint(0,1) for x in range(200)])
y_pred = np.array([randint(0,1) for x in range(200)])
confusion_matrix_test = confusion_matrix(y_true, y_pred)
base_utils = utils(confusion_matrix_test, y_true, y_pred)
base_utils.get_value()

{'Confusion Matrix': {'TP': 40,
  'TN': 55,
  'FN': 45,
  'FP': 45,
  'precision': 0.47058823529411764,
  'recall': 0.47058823529411764,
  'f1_score': 0.47058823529411764,
  'sensitivity': 0.47058823529411764,
  'specificity': 0.55,
  'negative_predictive_value': 0.55,
  'false_negative_rate': 0.5294117647058824,
  'false_positive_rate': 0.45,
  'false_discovery_rate': 0.5294117647058824,
  'false_omission_rate': 0.45,
  'Positive_likelihood_ratio': 1.0457516339869282,
  'Negative_likelihood_ratio': 0.9625668449197861,
  'prevalence_threshold': 0.4944082464090813,
  'threat_score': 0.27586206896551724,
  'Prevalence': 0.4594594594594595,
  'Matthews_correlation_coefficient': 2.4221453287197232e-06,
  'Fowlkes_Mallows_index': 0.9701425001453319,
  'informedness': 0.020588235294117574,
  'markedness': 0.020588235294117574,
  'Diagnostic_odds_ratio': 1.08641975308642,
  'accuracy': 0.5135135135135135,
  'balanced_accuracy': 0.5102941176470588},
 'ROC_AUC_SCORE': {'Macro': 0.47500000000000

In [10]:
class Training:
    def __init__(self, fold_count, X, y):
        self.fold_count = fold_count
        self.param_grid = {
            "criterion" : ["gini", "entropy", "log_loss"],
            "splitter" : ["best", "random"],
            "max_features" : ["sqrt", "log2"],
            "class_weight" : ["balanced", None]
        }
        self.kf = KFold(n_splits=fold_count)
        self.history = {}
        self.X = X
        self.y = y
    
    def get_fold_value(self):
        return self.kf        
        
    def training(self, criterion,
                 splitter, 
                 max_features, 
                 class_weight, train_case):
    
        # output_dict initilize
        output_dict = {}
    
        # poiter track the index of fold
        fold_index = 0
    
        for train_index, test_index in zip(self.X, self.y):
            train_df = pd.read_csv(train_index)
            test_df = pd.read_csv(test_index)
            
            print("\tFold: {}".format(fold_index))
            print("\tTRAIN:", train_index, "\n\tTEST:", test_index)
        
            # folding data
            x_train, y_train = train_df.iloc[:, 2:-1], train_df.iloc[:, -1]
            x_test, y_test = test_df.iloc[:, 2:-1], test_df.iloc[:, -1]
    
            # Training
            print("\t\tTraining : {}".format(fold_index), end = " -- ")
            print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end=" --- ")
            model = DecisionTreeClassifier(criterion = criterion, 
                                              splitter = splitter, 
                                              max_features = max_features, 
                                              class_weight = class_weight)
            model.fit(x_train,y_train)
            print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
            # Testing
            print("\t\tValidation: {}".format(fold_index), end = " -- ")
            print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end="---")
            y_pred = model.predict(x_test)
            print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
            # Evaluation
            cm = confusion_matrix(y_test, y_pred)
            current_utils = utils(cm, y_test, y_pred)
            output_dict["fold_{}".format(fold_index)] = current_utils.get_value()
            
            fold_index += 1
        print("\n")
        return output_dict 
    
    def train(self, path):
        count = 0
        for x in self.param_grid["criterion"]:
            for i in self.param_grid["splitter"]:
                for j in self.param_grid["max_features"]:
                    for k in self.param_grid["class_weight"]:
                        print("Traning Case: {}".format(count))
                        self.history["train_{}".format(count)] = {
                            "param" : {
                            "criterion" : x,
                            "splitter" : i,
                            "max_features" : j,
                            "class_weight" : k
                            },
                            "train_fold" : self.training(x, i, j, k, count)
                        }
                        count += 1
        with open(path.format(self.fold_count), 'wb') as f:
            pickle.dump(self.history, f)
        
        return self.history

# Training

In [11]:
training = Training(10, kmean_train_paths, kmean_test_paths)
approach_dir = os.getcwd() + "\\Experiment\\Approach"
svm_dir = approach_dir + "\\DecisionTree"
save_result_path = svm_dir + '\\rdf_km64_orb_result_fold{0}_0.9.pkl'
training.train(path = save_result_path)

Traning Case: 0
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index0.csv
		Training : 0 -- Start: 11/14/2022, 15:13:24 --- End: 11/14/2022, 15:13:24
		Validation: 0 -- Start: 11/14/2022, 15:13:24---End: 11/14/2022, 15:13:24
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index1.csv
		Training : 1 -- Start: 11/14/2022, 15:13:24 --- End: 11/14/2022, 15:13:24
		Validation: 1 -- Start: 11/14/2022, 15:13:24---End: 11/14/2022, 15:13:24
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20

C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv
		Training : 4 -- Start: 11/14/2022, 15:13:24 --- End: 11/14/2022, 15:13:24
		Validation: 4 -- Start: 11/14/2022, 15:13:24---End: 11/14/2022, 15:13:24
	Fold: 5
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv
		Training : 5 -- Start: 11/14/2022, 15:13:24 --- End: 11/14/2022, 15:13:24
		Validation: 5 -- Start: 11/14/2022, 15:13:24---End: 11/14/2022, 15:13:24
	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_inde

C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 5
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv
		Training : 5 -- Start: 11/14/2022, 15:13:25 --- End: 11/14/2022, 15:13:25
		Validation: 5 -- Start: 11/14/2022, 15:13:25---End: 11/14/2022, 15:13:25
	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index6.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index6.csv
		Training : 6 -- Start: 11/14/2022, 15:13:25 --- End: 11/14/2022, 15:13:25
		Validation: 6 -- Start: 11/14/2022, 15:13:25---End: 11/14/2022, 15:13:25
	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_inde

...

C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv
		Training : 4 -- Start: 11/14/2022, 15:13:29 --- End: 11/14/2022, 15:13:29
		Validation: 4 -- Start: 11/14/2022, 15:13:29---End: 11/14/2022, 15:13:29
	Fold: 5
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv
		Training : 5 -- Start: 11/14/2022, 15:13:29 --- End: 11/14/2022, 15:13:29
		Validation: 5 -- Start: 11/14/2022, 15:13:29---End: 11/14/2022, 15:13:29
	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_inde

C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index6.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index6.csv
		Training : 6 -- Start: 11/14/2022, 15:13:29 --- End: 11/14/2022, 15:13:29
		Validation: 6 -- Start: 11/14/2022, 15:13:29---End: 11/14/2022, 15:13:29
	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index7.csv
		Training : 7 -- Start: 11/14/2022, 15:13:29 --- End: 11/14/2022, 15:13:29
		Validation: 7 -- Start: 11/14/2022, 15:13:29---End: 11/14/2022, 15:13:29
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_inde

{'train_0': {'param': {'criterion': 'gini',
   'splitter': 'best',
   'max_features': 'sqrt',
   'class_weight': 'balanced'},
  'train_fold': {'fold_0': {'Confusion Matrix': {'TP': 5,
     'TN': 10,
     'FN': 2,
     'FP': 2,
     'precision': 0.7142857142857143,
     'recall': 0.7142857142857143,
     'f1_score': 0.7142857142857143,
     'sensitivity': 0.7142857142857143,
     'specificity': 0.8333333333333334,
     'negative_predictive_value': 0.8333333333333334,
     'false_negative_rate': 0.2857142857142857,
     'false_positive_rate': 0.16666666666666666,
     'false_discovery_rate': 0.2857142857142857,
     'false_omission_rate': 0.16666666666666666,
     'Positive_likelihood_ratio': 4.2857142857142865,
     'Negative_likelihood_ratio': 0.3428571428571428,
     'prevalence_threshold': 0.32571203244301905,
     'threat_score': 0.35714285714285715,
     'Prevalence': 0.3684210526315789,
     'Matthews_correlation_coefficient': 0.006519274376417234,
     'Fowlkes_Mallows_index': 1.

# Result Analyzing

In [12]:
file = open(save_result_path.format(10), 'rb')
result = pickle.load(file)
result

{'train_0': {'param': {'criterion': 'gini',
   'splitter': 'best',
   'max_features': 'sqrt',
   'class_weight': 'balanced'},
  'train_fold': {'fold_0': {'Confusion Matrix': {'TP': 5,
     'TN': 10,
     'FN': 2,
     'FP': 2,
     'precision': 0.7142857142857143,
     'recall': 0.7142857142857143,
     'f1_score': 0.7142857142857143,
     'sensitivity': 0.7142857142857143,
     'specificity': 0.8333333333333334,
     'negative_predictive_value': 0.8333333333333334,
     'false_negative_rate': 0.2857142857142857,
     'false_positive_rate': 0.16666666666666666,
     'false_discovery_rate': 0.2857142857142857,
     'false_omission_rate': 0.16666666666666666,
     'Positive_likelihood_ratio': 4.2857142857142865,
     'Negative_likelihood_ratio': 0.3428571428571428,
     'prevalence_threshold': 0.32571203244301905,
     'threat_score': 0.35714285714285715,
     'Prevalence': 0.3684210526315789,
     'Matthews_correlation_coefficient': 0.006519274376417234,
     'Fowlkes_Mallows_index': 1.

In [13]:
print("Number of Training: {}".format(len(list(result.keys()))))

Number of Training: 24


In [14]:
training_case_lst = list(result.keys())
print(training_case_lst[0])

train_0


In [15]:
train_sample = result[training_case_lst[0]]
print(train_sample['param'])
print("Number of Fold: {}".format(len(train_sample['train_fold'])))

{'criterion': 'gini', 'splitter': 'best', 'max_features': 'sqrt', 'class_weight': 'balanced'}
Number of Fold: 10


In [16]:
fold_sample = train_sample['train_fold']['fold_0']
print(fold_sample['Classification Report'])

{'0': {'precision': 0.7142857142857143, 'recall': 0.8333333333333334, 'f1-score': 0.7692307692307692, 'support': 6}, '1': {'precision': 0.9090909090909091, 'recall': 0.8333333333333334, 'f1-score': 0.8695652173913043, 'support': 12}, 'accuracy': 0.8333333333333334, 'macro avg': {'precision': 0.8116883116883117, 'recall': 0.8333333333333334, 'f1-score': 0.8193979933110367, 'support': 18}, 'weighted avg': {'precision': 0.8441558441558441, 'recall': 0.8333333333333334, 'f1-score': 0.8361204013377926, 'support': 18}}


In [17]:
result_analyze_dict = {
    'Train' : [],
    'MACC' : [],
    'MAP' : [],
    'MASens' : [],
    'MASpec' : [],
    'MAF1' : [],
    'AUC' : []
}

for result_key in result:
    result_analyze_dict['Train'].append(result_key)
    train_base = result[result_key]
    fold_base = train_base['train_fold']
    acc_lst, pre_lst, re_lst, spec_lst, f1_lst, auc_lst = [], [], [], [], [], []
    for fold_key in fold_base:
        current_fold = fold_base[fold_key]
        current_fold_cls = current_fold['Classification Report']
        acc_lst.append(current_fold_cls['accuracy'])
        pre_lst.append(current_fold_cls['macro avg']['precision'])
        re_lst.append(current_fold_cls['macro avg']['recall'])
        spec_lst.append(current_fold['Confusion Matrix']['specificity'])
        auc_lst.append(current_fold['ROC_AUC_SCORE']['Macro'])
        f1_lst.append(current_fold_cls['macro avg']['f1-score'])
    result_analyze_dict['MACC'].append(sum(acc_lst)/len(acc_lst))
    result_analyze_dict['MAP'].append(sum(pre_lst)/len(pre_lst))
    result_analyze_dict['MASens'].append(sum(re_lst)/len(re_lst))
    result_analyze_dict['MASpec'].append(sum(spec_lst)/len(spec_lst))
    result_analyze_dict['MAF1'].append(sum(f1_lst)/len(f1_lst))
    result_analyze_dict['AUC'].append(sum(auc_lst)/len(auc_lst))

result_df = pd.DataFrame(result_analyze_dict)
result_df.head()

,Train,MACC,MAP,MASens,MASpec,MAF1,AUC
0,train_0,0.773016,0.764165,0.754167,0.825000,0.731959,0.754167
1,train_1,0.776984,0.739739,0.720833,0.841667,0.721342,0.720833
2,train_2,0.734921,0.722612,0.695833,0.808333,0.689379,0.695833
3,train_3,0.757143,0.734000,0.712500,0.841667,0.703460,0.712500
4,train_4,0.764286,0.737773,0.712500,0.825000,0.714013,0.712500


In [18]:
result_max = result_df.max()
result_max

Train      train_9
MACC      0.830952
MAP       0.830021
MASens    0.791667
MASpec         0.9
MAF1      0.794917
AUC       0.791667
dtype: object

In [19]:
id_result_max = result_df.iloc[:, 1:].idxmax()
id_result_max

MACC      17
MAP       17
MASens    16
MASpec    17
MAF1      17
AUC       16
dtype: int64

In [20]:
id_max = 'train_{}'.format(id_result_max.mode()[0])
print(id_max)

train_17


In [21]:
result_df.loc[result_df['Train'] == id_max]

,Train,MACC,MAP,MASens,MASpec,MAF1,AUC
17,train_17,0.830952,0.830021,0.791667,0.9,0.794917,0.791667


In [22]:
train_max = result[id_max]
train_max['param']

{'criterion': 'log_loss',
 'splitter': 'best',
 'max_features': 'sqrt',
 'class_weight': None}

In [23]:
train_max_fold = train_max['train_fold']
train_max_analyze = {
    'Fold' : [],
    'ACC' : [],
    'AP' : [],
    'ASens' : [],
    'ASpec' : [],
    'AF1' : [],
    'AUC' : []
}

for fold_key in train_max_fold:
    train_max_analyze['Fold'].append(fold_key)
    current_fold = train_max_fold[fold_key]
    current_fold_cls = current_fold['Classification Report']
    train_max_analyze['ACC'].append(current_fold_cls['accuracy'])
    train_max_analyze['AP'].append(current_fold_cls['macro avg']['precision'])
    train_max_analyze['ASens'].append(current_fold_cls['macro avg']['recall'])
    train_max_analyze['ASpec'].append(current_fold['Confusion Matrix']['specificity'])
    train_max_analyze['AF1'].append(current_fold_cls['macro avg']['f1-score'])
    train_max_analyze['AUC'].append(current_fold['ROC_AUC_SCORE']['Macro'])

train_max_analyze_df = pd.DataFrame(train_max_analyze)
train_max_analyze_df

,Fold,ACC,AP,ASens,ASpec,AF1,AUC
0,fold_0,0.833333,0.900000,0.750000,1.000000,0.777778,0.750000
1,fold_1,0.833333,0.900000,0.750000,1.000000,0.777778,0.750000
2,fold_2,0.944444,0.961538,0.916667,1.000000,0.934545,0.916667
3,fold_3,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
4,fold_4,0.666667,0.650000,0.666667,0.666667,0.649351,0.666667
5,fold_5,0.944444,0.961538,0.916667,1.000000,0.934545,0.916667
6,fold_6,0.722222,0.684615,0.666667,0.833333,0.672727,0.666667
7,fold_7,0.888889,0.875000,0.875000,0.916667,0.875000,0.875000
8,fold_8,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9,fold_9,0.642857,0.544444,0.583333,0.666667,0.523810,0.583333


# N_times_K_folds

In [24]:
# Setup

train_max_param = train_max['param']
n_times = 100

nt_kf_output_dict = {
    'Train' : [], # <index>_<fold>
    'ACC' : [],
    'AP' : [],
    'ASens' : [],
    'ASpec' : [],
    'AF1' : [],
    'AUC' : []
}

for n in range(n_times):
    print("Training {}".format(n))
    fold_index = 0
    for train_index, test_index in zip(kmean_train_paths, kmean_test_paths):
        train_df = pd.read_csv(train_index)
        test_df = pd.read_csv(test_index)      
    
        print("\tFold: {}".format(fold_index))
        print("\tTRAIN:", train_index, "\n\tTEST:", test_index)

        # folding data
        x_train, y_train = train_df.iloc[:, 2:-1], train_df.iloc[:, -1]
        x_test, y_test = test_df.iloc[:, 2:-1], test_df.iloc[:, -1]
        
        # Training
        print("\t\tTraining : {}".format(fold_index), end = " -- ")
        print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end=" --- ")
        model = DecisionTreeClassifier(criterion = train_max_param['criterion'], 
                                      splitter = train_max_param['splitter'], 
                                      max_features = train_max_param['max_features'], 
                                      class_weight = train_max_param['class_weight'])
        model.fit(x_train, y_train)
        print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))

        # Testing
        print("\t\tValidation: {}".format(fold_index), end = " -- ")
        print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end="---")
        y_pred = model.predict(x_test)
        print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
        # Evaluation
        cm = confusion_matrix(y_test, y_pred)
        base_result = utils(cm, y_test, y_pred).get_value()
        
        # Result Storing
        nt_kf_output_dict['Train'].append('{0}_{1}'.format(n, fold_index))
        cls = base_result['Classification Report']
        nt_kf_output_dict['ACC'].append(cls['accuracy'])
        nt_kf_output_dict['AP'].append(cls['macro avg']['precision'])
        nt_kf_output_dict['ASens'].append(cls['macro avg']['recall'])
        nt_kf_output_dict['ASpec'].append(base_result['Confusion Matrix']['specificity'])
        nt_kf_output_dict['AF1'].append(cls['macro avg']['f1-score'])
        nt_kf_output_dict['AUC'].append(base_result['ROC_AUC_SCORE']['Macro'])
        
        # Fold index increment
        fold_index += 1

Training 0
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index0.csv
		Training : 0 -- Start: 11/14/2022, 15:13:30 --- End: 11/14/2022, 15:13:30
		Validation: 0 -- Start: 11/14/2022, 15:13:30---End: 11/14/2022, 15:13:30
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index1.csv
		Training : 1 -- Start: 11/14/2022, 15:13:30 --- End: 11/14/2022, 15:13:30
		Validation: 1 -- Start: 11/14/2022, 15:13:30---End: 11/14/2022, 15:13:30
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo

C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25

	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index1.csv
		Training : 1 -- Start: 11/14/2022, 15:13:31 --- End: 11/14/2022, 15:13:31
		Validation: 1 -- Start: 11/14/2022, 15:13:31---End: 11/14/2022, 15:13:31
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index2.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index2.csv
		Training : 2 -- Start: 11/14/2022, 15:13:31 --- End: 11/14/2022, 15:13:31
		Validation: 2 -- Start: 11/14/2022, 15:13:31---End: 11/14/2022, 15:13:31
	Fold: 3
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_inde

C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 3
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index3.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index3.csv
		Training : 3 -- Start: 11/14/2022, 15:13:31 --- End: 11/14/2022, 15:13:31
		Validation: 3 -- Start: 11/14/2022, 15:13:31---End: 11/14/2022, 15:13:31
	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv
		Training : 4 -- Start: 11/14/2022, 15:13:31 --- End: 11/14/2022, 15:13:31
		Validation: 4 -- Start: 11/14/2022, 15:13:31---End: 11/14/2022, 15:13:31
	Fold: 5
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_inde

...

	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index4.csv
		Training : 4 -- Start: 11/14/2022, 15:13:48 --- End: 11/14/2022, 15:13:48
		Validation: 4 -- Start: 11/14/2022, 15:13:48---End: 11/14/2022, 15:13:48
	Fold: 5
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index5.csv
		Training : 5 -- Start: 11/14/2022, 15:13:48 --- End: 11/14/2022, 15:13:48
		Validation: 5 -- Start: 11/14/2022, 15:13:48---End: 11/14/2022, 15:13:48
	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_inde

C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index6.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index6.csv
		Training : 6 -- Start: 11/14/2022, 15:13:49 --- End: 11/14/2022, 15:13:49
		Validation: 6 -- Start: 11/14/2022, 15:13:49---End: 11/14/2022, 15:13:49
	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index7.csv
		Training : 7 -- Start: 11/14/2022, 15:13:49 --- End: 11/14/2022, 15:13:49
		Validation: 7 -- Start: 11/14/2022, 15:13:49---End: 11/14/2022, 15:13:49
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_inde

C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_5696\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index7.csv
		Training : 7 -- Start: 11/14/2022, 15:13:49 --- End: 11/14/2022, 15:13:49
		Validation: 7 -- Start: 11/14/2022, 15:13:49---End: 11/14/2022, 15:13:49
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_index8.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_test_10000_1.2_8_31_0_2_31_20histo_64_cv_index8.csv
		Training : 8 -- Start: 11/14/2022, 15:13:49 --- End: 11/14/2022, 15:13:49
		Validation: 8 -- Start: 11/14/2022, 15:13:49---End: 11/14/2022, 15:13:49
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\orb_train_10000_1.2_8_31_0_2_31_20histo_64_cv_inde

In [25]:
nt_kf_output_df = pd.DataFrame(nt_kf_output_dict)
nt_kf_output_df.head(10)

,Train,ACC,AP,ASens,ASpec,AF1,AUC
0,0_0,0.833333,0.900000,0.750000,1.000000,0.777778,0.750000
1,0_1,0.833333,0.900000,0.750000,1.000000,0.777778,0.750000
2,0_2,0.833333,0.900000,0.750000,1.000000,0.777778,0.750000
3,0_3,0.888889,0.928571,0.833333,1.000000,0.861538,0.833333
4,0_4,0.555556,0.500000,0.500000,0.666667,0.500000,0.500000
5,0_5,0.777778,0.767857,0.708333,0.916667,0.723077,0.708333
6,0_6,0.777778,0.762500,0.791667,0.750000,0.766234,0.791667
7,0_7,0.777778,0.875000,0.666667,1.000000,0.678571,0.666667
8,0_8,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
9,0_9,0.714286,0.575000,0.625000,0.750000,0.575758,0.625000


In [26]:
nt_kf_output_df.mean()

Train         inf
ACC      0.763690
AP       0.743625
ASens    0.720208
ASpec    0.838750
AF1      0.714019
AUC      0.720208
dtype: float64

In [27]:
nt_kf_output_df.std(numeric_only=True)

ACC      0.106737
AP       0.137944
ASens    0.126937
ASpec    0.129990
AF1      0.133145
AUC      0.126937
dtype: float64

In [28]:
acc_values = nt_kf_output_df['ACC'].values.tolist()
ap_values = nt_kf_output_df['AP'].values.tolist()
asens_values = nt_kf_output_df['ASens'].values.tolist()
aspec_values = nt_kf_output_df['ASpec'].values.tolist()
af1_values = nt_kf_output_df['AF1'].values.tolist()
auc_values = nt_kf_output_df['AUC'].values.tolist()
print(len(acc_values), len(ap_values), len(asens_values), len(aspec_values), len(af1_values), len(auc_values))

1000 1000 1000 1000 1000 1000


In [29]:
import scipy.stats as st

def CI(data, confidence_level, dis_type = 't'):
    if dis_type == 't':
        return st.t.interval(confidence=confidence_level, df=len(data)-1, 
                             loc=np.mean(data), scale=st.sem(data))
    elif dis_type == 'g':
        return st.norm.interval(confidence=confidence_level, loc=np.mean(data), scale=st.sem(data))

def CI_calculator(confidence_level, dis_type = 't'):
    for x in nt_kf_output_df.columns[1:]:
        base = nt_kf_output_df[x].values.tolist()
        print("CI of {}".format(x), CI(base, confidence_level, dis_type))

In [30]:
cl = 0.95

CI_calculator(confidence_level = cl, dis_type = 't')

CI of ACC (0.7570669203230864, 0.7703140320578659)
CI of AP (0.7350646124007575, 0.752184720864017)
CI of ASens (0.7123313117390514, 0.7280853549276153)
CI of ASpec (0.8306835402654935, 0.8468164597345065)
CI of AF1 (0.7057568707561966, 0.7222814747269856)
CI of AUC (0.7123313117390514, 0.7280853549276153)


In [31]:
cl = 0.95

CI_calculator(confidence_level = cl, dis_type = 'g')

CI of ACC (0.7570749450982602, 0.770306007282692)
CI of AP (0.7350749833422969, 0.7521743499224776)
CI of ASens (0.7123408551517738, 0.7280758115148929)
CI of ASpec (0.8306933131921682, 0.8468066868078318)
CI of AF1 (0.7057668809557296, 0.7222714645274526)
CI of AUC (0.7123408551517738, 0.7280758115148929)


In [32]:
cl = 0.99

CI_calculator(confidence_level = cl, dis_type = 'g')

CI of ACC (0.7549961945708741, 0.7723847578100782)
CI of AP (0.7323884781470947, 0.7548608551176798)
CI of ASens (0.7098687143378097, 0.7305479523288569)
CI of ASpec (0.8281617187189033, 0.8493382812810967)
CI of AF1 (0.7031738229437752, 0.724864522539407)
CI of AUC (0.7098687143378097, 0.7305479523288569)
